In [5]:
!pip install cvzone mediapipe

In [1]:
#final code full final it is

import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot

# Open webcam
cap = cv2.VideoCapture(0)  
detector = FaceMeshDetector(maxFaces=1)
plotY = LivePlot(640, 360, [20, 50], invert=True)

idList = [22, 23, 24, 26, 110, 157, 158, 159, 160, 161, 130, 243]
ratioList = []
blinkCounter = 0
counter = 0
color = (255, 0, 255)

# Parameters
smooth_window = 7       # number of frames for smoothing
blink_threshold = 32    # threshold for closed eyes (tune if needed)
min_closed_frames = 3   # must stay closed this many frames to count as blink

while True:
    success, img = cap.read()
    if not success:
        print("Failed to grab frame")
        break

    img, faces = detector.findFaceMesh(img, draw=False)

    if faces:
        face = faces[0]

        # Eye landmarks
        leftUp, leftDown = face[159], face[23]
        leftLeft, leftRight = face[130], face[243]

        # Distances
        lengthVer, _ = detector.findDistance(leftUp, leftDown)
        lengthHor, _ = detector.findDistance(leftLeft, leftRight)

        # EAR-like ratio
        ratio = (lengthVer / (lengthHor + 1e-6)) * 100
        ratioList.append(ratio)
        if len(ratioList) > smooth_window:
            ratioList.pop(0)
        ratioAvg = sum(ratioList) / len(ratioList)

        # Blink detection with stricter logic
        if ratioAvg < blink_threshold:
            counter += 1
        else:
            if counter >= min_closed_frames:
                blinkCounter += 1
                color = (0, 200, 0)
            counter = 0

        # Show blink count
        cvzone.putTextRect(img, f'Blink Count: {blinkCounter}', (50, 100), colorR=color)

        # Plot
        imgPlot = plotY.update(ratioAvg, color)
        img = cv2.resize(img, (640, 360))
        imgStack = cvzone.stackImages([img, imgPlot], 2, 1)
    else:
        img = cv2.resize(img, (640, 360))
        imgStack = cvzone.stackImages([img, img], 2, 1)

    cv2.imshow("Eye Blink Detection - Improved", imgStack)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [53]:
# the first proper model

import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot

# 🔹 Open webcam instead of video file
cap = cv2.VideoCapture(0)  
detector = FaceMeshDetector(maxFaces=1)
plotY = LivePlot(640, 360, [20, 50], invert=True)

idList = [22, 23, 24, 26, 110, 157, 158, 159, 160, 161, 130, 243]
ratioList = []
blinkCounter = 0
counter = 0
color = (255, 0, 255)

while True:
    success, img = cap.read()
    if not success:
        print("Failed to grab frame")
        break

    img, faces = detector.findFaceMesh(img, draw=False)

    if faces:
        face = faces[0]
        for id in idList:
            cv2.circle(img, face[id], 5, color, cv2.FILLED)

        leftUp = face[159]
        leftDown = face[23]
        leftLeft = face[130]
        leftRight = face[243]
        lengthVer, _ = detector.findDistance(leftUp, leftDown)
        lengthHor, _ = detector.findDistance(leftLeft, leftRight)

        cv2.line(img, leftUp, leftDown, (0, 200, 0), 3)
        cv2.line(img, leftLeft, leftRight, (0, 200, 0), 3)

        ratio = int((lengthVer / lengthHor) * 100)
        ratioList.append(ratio)
        if len(ratioList) > 3:
            ratioList.pop(0)
        ratioAvg = sum(ratioList) / len(ratioList)

        if ratioAvg < 35 and counter == 0:
            blinkCounter += 1
            color = (0, 200, 0)
            counter = 1
        if counter != 0:
            counter += 1
            if counter > 10:
                counter = 0
                color = (255, 0, 255)

        cvzone.putTextRect(img, f'Blink Count: {blinkCounter}', (50, 100), colorR=color)

        imgPlot = plotY.update(ratioAvg, color)
        img = cv2.resize(img, (640, 360))
        imgStack = cvzone.stackImages([img, imgPlot], 2, 1)
    else:
        img = cv2.resize(img, (640, 360))
        imgStack = cvzone.stackImages([img, img], 2, 1)

    cv2.imshow("Eye Blink Detection - Webcam", imgStack)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press Q to exit
        break

cap.release()
cv2.destroyAllWindows()


In [39]:
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot

# Open webcam
cap = cv2.VideoCapture(0)  
detector = FaceMeshDetector(maxFaces=1)
plotY = LivePlot(640, 360, [20, 50], invert=True)

# FaceMesh IDs for left eye
idList = [22, 23, 24, 26, 110, 157, 158, 159, 160, 161, 130, 243]

ratioList = []
blinkCounter = 0
counter = 0
color = (255, 0, 255)

# Calibration variables
calibration_frames = 20
open_eye_baseline = None
lower_thresh = None
upper_thresh = None

while True:
    success, img = cap.read()
    if not success:
        print("Failed to grab frame")
        break

    img, faces = detector.findFaceMesh(img, draw=False)

    if faces:
        face = faces[0]

        # Draw eye landmarks
        for id in idList:
            cv2.circle(img, face[id], 3, color, cv2.FILLED)

        # Key eye points
        leftUp = face[159]
        leftDown = face[23]
        leftLeft = face[130]
        leftRight = face[243]

        # Distances
        lengthVer, _ = detector.findDistance(leftUp, leftDown)
        lengthHor, _ = detector.findDistance(leftLeft, leftRight)

        # Ratio (EAR-like)
        ratio = (lengthVer / (lengthHor + 1e-6)) * 100
        ratioList.append(ratio)
        if len(ratioList) > 5:
            ratioList.pop(0)
        ratioAvg = sum(ratioList) / len(ratioList)

        # ---------------- Calibration Stage ----------------
        if open_eye_baseline is None and len(ratioList) >= calibration_frames:
            open_eye_baseline = sum(ratioList) / len(ratioList)
            lower_thresh = open_eye_baseline * 0.75   # 75% of open-eye value
            upper_thresh = open_eye_baseline * 0.90   # 90% of open-eye value
            print(f"Calibration done ✅ Open eye baseline: {open_eye_baseline:.2f}")
            print(f"Thresholds → Lower: {lower_thresh:.2f}, Upper: {upper_thresh:.2f}")

        # ---------------- Blink Detection ----------------
        if open_eye_baseline is not None:
            if ratioAvg < lower_thresh and counter == 0:
                blinkCounter += 1
                color = (0, 200, 0)
                counter = 1
            if counter != 0:
                counter += 1
                if ratioAvg > upper_thresh and counter > 10:
                    counter = 0
                    color = (255, 0, 255)

        # Display blink count
        cvzone.putTextRect(img, f'Blink Count: {blinkCounter}', (50, 100), colorR=color)

        # Plot ratio curve
        imgPlot = plotY.update(ratioAvg, color)
        img = cv2.resize(img, (640, 360))
        imgStack = cvzone.stackImages([img, imgPlot], 2, 1)
    else:
        img = cv2.resize(img, (640, 360))
        imgStack = cvzone.stackImages([img, img], 2, 1)

    cv2.imshow("Eye Blink Detection - Auto Calibrated", imgStack)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [35]:
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot

cap = cv2.VideoCapture(0)
detector = FaceMeshDetector(maxFaces=1)
plotY = LivePlot(640, 360, [20, 50], invert=True)

idList = [22, 23, 24, 26, 110, 157, 158, 159, 160, 161, 130, 243]
ratioList = []
blinkCounter = 0
counter = 0
color = (255, 0, 255)

lower_thresh = 30   # adjust these after testing
upper_thresh = 38

while True:
    success, img = cap.read()
    if not success:
        break

    img, faces = detector.findFaceMesh(img, draw=False)

    if faces:
        face = faces[0]
        leftUp = face[159]
        leftDown = face[23]
        leftLeft = face[130]
        leftRight = face[243]
        lengthVer, _ = detector.findDistance(leftUp, leftDown)
        lengthHor, _ = detector.findDistance(leftLeft, leftRight)

        ratio = (lengthVer / (lengthHor + 1e-6)) * 100
        ratioList.append(ratio)
        if len(ratioList) > 5:
            ratioList.pop(0)
        ratioAvg = sum(ratioList) / len(ratioList)

        print("EAR Ratio:", ratioAvg)  # 👈 Check your open/closed eye values

        if ratioAvg < lower_thresh and counter == 0:
            blinkCounter += 1
            color = (0, 200, 0)
            counter = 1
        if counter != 0:
            counter += 1
            if ratioAvg > upper_thresh and counter > 10:
                counter = 0
                color = (255, 0, 255)

        cvzone.putTextRect(img, f'Blink Count: {blinkCounter}', (50, 100), colorR=color)
        imgPlot = plotY.update(ratioAvg, color)
        img = cv2.resize(img, (640, 360))
        imgStack = cvzone.stackImages([img, imgPlot], 2, 1)
    else:
        img = cv2.resize(img, (640, 360))
        imgStack = cvzone.stackImages([img, img], 2, 1)

    cv2.imshow("Eye Blink Detection", imgStack)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


EAR Ratio: 22.70382973034895
EAR Ratio: 27.93586748528597
EAR Ratio: 29.35429097776169
EAR Ratio: 30.904786742127477
EAR Ratio: 31.19416804625418
EAR Ratio: 33.12374075273659
EAR Ratio: 32.92838729723461
EAR Ratio: 32.960498357244184
EAR Ratio: 32.50114438133768
EAR Ratio: 32.469033321328105
EAR Ratio: 32.446953263067314
EAR Ratio: 32.042297840881375
EAR Ratio: 32.9192238013744
EAR Ratio: 32.95240056993317
EAR Ratio: 33.873659275126435
EAR Ratio: 34.50745103263192
EAR Ratio: 35.82114347532047
EAR Ratio: 35.55592921407212
EAR Ratio: 35.292464398868695
EAR Ratio: 34.403316753685
EAR Ratio: 33.75949399443074
EAR Ratio: 32.68116935904516
EAR Ratio: 32.06945765980046
EAR Ratio: 32.10148585242291
EAR Ratio: 31.884897090310982
EAR Ratio: 32.29671139640163
EAR Ratio: 32.87275650992767
EAR Ratio: 33.46619192031865
EAR Ratio: 33.885762896785174
EAR Ratio: 34.714063358141786
EAR Ratio: 34.97527626655348
EAR Ratio: 34.56851876791042
EAR Ratio: 34.18982230466447
EAR Ratio: 33.738223135575495
EAR Ra

In [20]:
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot

cap = cv2.VideoCapture(0)  # webcam
detector = FaceMeshDetector(maxFaces=1)
plotY = LivePlot(640, 360, [20, 50], invert=True)

idList = [22, 23, 24, 26, 110, 157, 158, 159, 160, 161, 130, 243]
ratioList = []
blinkCounter = 0
counter = 0
color = (255, 0, 255)

# New parameters
warmup_frames = 30  # number of frames to skip counting
frame_count = 0
lower_thresh = 30   # threshold to detect blink
upper_thresh = 40   # threshold to exit blink state (hysteresis)

while True:
    success, img = cap.read()
    if not success:
        print("Failed to grab frame")
        break

    frame_count += 1

    img, faces = detector.findFaceMesh(img, draw=False)

    if faces:
        face = faces[0]
        # draw the landmarks
        for id in idList:
            cv2.circle(img, face[id], 5, color, cv2.FILLED)

        leftUp = face[159]
        leftDown = face[23]
        leftLeft = face[130]
        leftRight = face[243]
        lengthVer, _ = detector.findDistance(leftUp, leftDown)
        lengthHor, _ = detector.findDistance(leftLeft, leftRight)

        cv2.line(img, leftUp, leftDown, (0, 200, 0), 3)
        cv2.line(img, leftLeft, leftRight, (0, 200, 0), 3)

        # Compute ratio
        ratio = (lengthVer / (lengthHor + 1e-6)) * 100  # avoid division by zero
        ratioList.append(ratio)
        if len(ratioList) > 5:
            ratioList.pop(0)

        # Only start average after warm-up and when ratioList is full
        if frame_count > warmup_frames and len(ratioList) == 5:
            ratioAvg = sum(ratioList) / len(ratioList)

            # Blink detection with hysteresis
            if ratioAvg < lower_thresh and counter == 0:
                blinkCounter += 1
                color = (0, 200, 0)
                counter = 1
            # Maintain blink state until ratioAvg goes above upper threshold
            if counter != 0:
                counter += 1
                if ratioAvg > upper_thresh and counter > 10:
                    counter = 0
                    color = (255, 0, 255)

            cvzone.putTextRect(img, f'Blink Count: {blinkCounter}', (50, 100), colorR=color)
            # Plot graph etc.
            imgPlot = plotY.update(ratioAvg, color)
            img = cv2.resize(img, (640, 360))
            imgStack = cvzone.stackImages([img, imgPlot], 2, 1)
        else:
            # If warming up, just show initial without blink count
            img = cv2.resize(img, (640, 360))
            imgStack = cvzone.stackImages([img, img], 2, 1)
    else:
        img = cv2.resize(img, (640, 360))
        imgStack = cvzone.stackImages([img, img], 2, 1)

    cv2.imshow("Eye Blink Detection - Webcam", imgStack)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot

print("cvzone and modules are working!")
